In [1]:
LOAD_MODEL = 'models/attention.01.pt'
MODEL_CHECKPOINT = 'models/attention.01.pt'
DATASET_PATH = 'data/interim/preprocessed_paranmt.tsv'

In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader

import os
os.chdir("..") # go to the root dir

# Get the Dataset

In [3]:
MAX_SENT_SIZE = 10
MAX_TOKENS = 8_000

In [4]:
from src.data.make_dataset import ParanmtDataset

train_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    train=True,
    seed=42,
)

In [5]:
train_dataset.build_vocab(
    min_freq=2,
    specials=['<unk>', '<pad>', '<sos>', '<eos>'],
    max_tokens=MAX_TOKENS,
)

In [6]:
enc_vocab = train_dataset.toxic_vocab
dec_vocab = train_dataset.neutral_vocab

print("size of encoder vocab:", len(enc_vocab))
print("size of decoder vocab:", len(dec_vocab))

size of encoder vocab: 8000
size of decoder vocab: 8000


In [7]:
val_dataset = ParanmtDataset(
    path=DATASET_PATH,
    max_sent_size=MAX_SENT_SIZE,
    vocabs=(enc_vocab, dec_vocab), # avoid data leakage
    train=False,
    seed=42,
    take_first=10_000,
)

In [8]:
train_dataset.df

,similarity,lenght_diff,toxic_sent,neutral_sent,toxic_val,neutral_val
0,0.708038,0.171429,"[what, the, hell, i, danger, looking, at, ?]","[what, the, hell, is, safe, watch, ?]",0.888703,0.130954
1,0.606822,0.238095,"[lisa, ,, hit, him, again, .]","[lisa, ,, one, more, .]",0.957538,0.000053
2,0.719271,0.051282,"[what, are, you, doing, with, that, hooker, ?]","[what, are, you, doing, with, that, outsider, ?]",0.998877,0.000056
3,0.821008,0.047619,"[we, are, going, to, hit, him, !]","[it, is, going, to, hit, !]",0.997299,0.014387
4,0.725030,0.096774,"[i, do, not, fucking, believe, it, !]","[i, do, not, freaking, believe, it]",0.957814,0.056393
...,...,...,...,...,...,...
157735,0.827812,0.200000,"[I, will, make, you, fall, !]","[I, am, going, to, fall, !]",0.590488,0.006672
157736,0.625040,0.333333,"[i, fucking, my, girlfriend, .]","[satisfying, my, girl, behind, my, back, .]",0.999578,0.029578
157737,0.815115,0.041667,"[he, is, going, to, shoot, again, .]","[he, is, going, to, fire, again, .]",0.989201,0.008294
157738,0.866068,0.037037,"[oh, ,, mars, solid, ,, you, stink, .]","[oh, ,, mars, solid, ,, you, smell, .]",0.999077,0.072257


In [9]:
len(train_dataset), len(val_dataset)

(157740, 10000)

# Build the Dataloaders

In [10]:
batch_size = 128

In [11]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
)

In [12]:
# let's check if shape and everything is ok
for batch in train_dataloader:
    toxic_sent, neutral_sent = batch
    print("toxic_sent.shape:", toxic_sent.shape)
    print("neutral_sent.shape:", neutral_sent.shape)
    break

toxic_sent.shape: torch.Size([128, 10])
neutral_sent.shape: torch.Size([128, 10])


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Load the Model

- EncoderDecoder (Seq2Seq) with Attention mechanism

In [14]:
from src.models.attention.encoder import Encoder
from src.models.attention.decoder import Decoder
from src.models.attention.attention import Attention
from src.models.attention import Seq2SeqAttention

In [15]:
## Encoder
enc_input_dim = len(enc_vocab)
enc_embed_dim = 128
enc_hidden_dim = 256
enc_dropout = 0.5
enc_padding_idx = enc_vocab['<pad>']

## Decoder
dec_output_dim = len(dec_vocab)
dec_embed_dim = 128
dec_hidden_dim = 256
dec_dropout = 0.5
dec_padding_idx = dec_vocab['<pad>']

In [16]:
# load the encoder and decoder for our model
encoder = Encoder(
    input_dim=enc_input_dim,
    embed_dim=enc_embed_dim,
    hidden_dim=enc_hidden_dim,
    dec_hidden_dim=dec_hidden_dim,
    dropout=enc_dropout,
    vocab=enc_vocab,
    padding_idx=enc_padding_idx,
).to(device)

attention = Attention(
    enc_hidden_dim,
    dec_hidden_dim,
)

decoder = Decoder(
    output_dim=dec_output_dim,
    embed_dim=dec_embed_dim,
    hidden_dim=dec_hidden_dim,
    attention=attention,
    enc_hidden_dim=enc_hidden_dim,
    dropout=dec_dropout,
    vocab=dec_vocab,
    padding_idx=dec_padding_idx,
).to(device)

In [17]:
best_loss = float('inf')

model = Seq2SeqAttention(
    encoder=encoder,
    decoder=decoder,
    device=device,
    max_sent_size=MAX_SENT_SIZE,
).to(device)

In [18]:
from src.models.utils import count_parameters

print(f"number of parameters in model: {count_parameters(model)//1e6}M")

number of parameters in model: 11.0M


In [19]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
criterion = torch.nn.CrossEntropyLoss(ignore_index=dec_vocab['<pad>'])

In [20]:
from src.models.train_model import train

best_loss = train(
    model=model,
    loaders=(train_dataloader, val_dataloader),
    optimizer=optimizer,
    criterion=criterion,
    epochs=20,
    device=device,
    best_loss=best_loss,
    ckpt_path=MODEL_CHECKPOINT,
    clip_grad=1,
    teacher_force={
        'value': 0.9,
        'gamma': 1.0,
        'update_every_n_epoch': 50,
    } # first 10 epoch teacher force 1, after it will be turned off
)

Evaluating 20: 100%|██████████| 79/79 [00:00<00:00, 115.38it/s, loss=3.6] 


In [21]:
# let's load the model and predict
model = torch.load(MODEL_CHECKPOINT)
model.to(device)
model.eval()

Seq2SeqAttention(
  (encoder): Encoder(
    (vocab): Vocab()
    (embedding): Embedding(8000, 128, padding_idx=1)
    (rnn): LSTM(128, 256, batch_first=True, bidirectional=True)
    (fc_hidden): Linear(in_features=512, out_features=256, bias=True)
    (fc_cell): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (vocab): Vocab()
    (embedding): Embedding(8000, 128, padding_idx=1)
    (rnn): LSTM(640, 256, batch_first=True)
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (dropout): Dropout(p=0.5, inplace=False)
    (fc_out): Linear(in_features=896, out_features=8000, bias=True)
  )
)

In [22]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenizer = TreebankWordDetokenizer()

# let's see how our model works
num_examples = 10
num_sentence = 3
dataset = train_dataset
for _ in range(num_examples):
    idx = np.random.randint(0, len(dataset))
    toxic_sent = detokenizer.detokenize(dataset.df.loc[idx, 'toxic_sent'])
    neutral_sent = detokenizer.detokenize(dataset.df.loc[idx, 'neutral_sent'])
    
    print('toxic_sent:', toxic_sent)
    print('neutral_sent:', neutral_sent)
    preds = model.predict(
        toxic_sent,
        beam=True,
        beam_search_num_candidates=num_sentence,
        post_process_text=False,
    ) # let's use beam search
    print("predictions:")
    for i in range(num_sentence):
        print(f"\t{i+1})", preds[i])
    print("\n")

toxic_sent: hit him.
neutral_sent: go get him.
predictions:
	1) ['hit', 'him', '.', '<eos>']
	2) ['beat', 'him', '.', '<eos>']
	3) ['hit', 'him', '!', '<eos>']


toxic_sent: do not be a pussy.
neutral_sent: do not be lame.
predictions:
	1) ['do', 'not', 'be', 'a', 'chicken', '.', '<eos>']
	2) ['do', 'not', 'be', 'a', 'quitter', '.', '<eos>']
	3) ['do', 'not', 'be', 'a', 'baby', '.', '<eos>']


toxic_sent: why do not you shut him up?
neutral_sent: why do not you close it down?
predictions:
	1) ['why', 'do', 'not', 'you', 'shut', 'him', '?', '<eos>']
	2) ['why', 'do', 'not', 'you', 'shut', 'him', 'up', '?', '<eos>']
	3) ['why', 'do', 'not', 'you', 'keep', 'him', 'up', '?', '<eos>']


toxic_sent: cats are bad doctors.
neutral_sent: cats make terrible doctors.
predictions:
	1) ['cats', 'are', 'wrong', '.', '<eos>']
	2) ['cats', 'are', 'wrong', 'doctors', '.', '<eos>']
	3) ['cats', 'are', 'bad', '.', '<eos>']


toxic_sent: the world can die tomorrow.
neutral_sent: the world could end tomorr